# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import sqlite3
import pandas as pd
conn = sqlite3.connect("data.sqlite")

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [25]:
# Your code here
pd.read_sql("""
SELECT firstName, lastName, city
FROM employees
join offices
on employees.officeCode = offices.officeCode
WHERE offices.city IS "Boston"
""", conn)

,firstName,lastName,city
0,Julie,Firrelli,Boston
1,Steve,Patterson,Boston


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [23]:
# Your code here
pd.read_sql("""
SELECT offices.officeCode, city, COUNT(*) AS number_of_employees
FROM employees
join offices
on employees.officeCode = offices.officeCode
GROUP BY offices.officeCode
""", conn)

,officeCode,city,number_of_employees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2


## Write 3 questions of your own and answer them

In [30]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""
pd.read_sql("""
SELECT offices.officeCode, city, COUNT(*) AS number_of_employees
FROM employees
join offices
on employees.officeCode = offices.officeCode
GROUP BY offices.officeCode
""", conn)

,officeCode,city,number_of_employees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2


In [35]:
"""
What are the total sales (amount) made by each customer?
"""
q = """
SELECT customers.customerName, SUM(payments.amount) AS totalSales
FROM customers
JOIN payments ON customers.customerNumber = payments.customerNumber
GROUP BY customers.customerName
ORDER BY SUM(payments.amount) DESC;
"""
pd.read_sql(q, conn)
# Your code here

,customerName,totalSales
0,Euro+ Shopping Channel,715738.98
1,Mini Gifts Distributors Ltd.,584188.24
2,"Australian Collectors, Co.",180585.07
3,Muscle Machine Inc,177913.95
4,"Dragon Souveniers, Ltd.",156251.03
...,...,...
93,Royale Belge,29217.18
94,Frau da Collezione,25358.32
95,Atelier graphique,22314.36
96,Auto-Moto Classics Inc.,21554.26


In [39]:
"""
Which products have never been ordered?
"""
pd.read_sql("""
SELECT products.productCode, products.productName
FROM products
LEFT JOIN orderdetails ON products.productCode = orderdetails.productCode
WHERE orderdetails.orderNumber IS NULL;
""",  conn)

# Your code here

,productCode,productName
0,S18_3233,1985 Toyota Supra


In [40]:
"""
What is the average payment made by customers?"""
query = """SELECT AVG(amount) AS averagePayment
FROM payments;
"""
pd.read_sql(query,conn)
# Your code here

,averagePayment
0,32431.645531


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [44]:
# Your code here
pd.read_sql("""
SELECT employees.firstName, employees.lastName, products.productName
FROM employees
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders ON customers.customerNumber = orders.customerNumber
JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber
JOIN products ON orderdetails.productCode = products.productCode;

""", conn)

,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang
...,...,...,...
2991,Martin,Gerard,1954 Greyhound Scenicruiser
2992,Martin,Gerard,1950's Chicago Surface Lines Streetcar
2993,Martin,Gerard,Diamond T620 Semi-Skirted Tanker
2994,Martin,Gerard,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [45]:
# Your code here
pd.read_sql("""
SELECT employees.firstName, employees.lastName, SUM(orderdetails.quantityOrdered) AS totalProductsSold
FROM employees
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders ON customers.customerNumber = orders.customerNumber
JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber
GROUP BY employees.employeeNumber, employees.firstName, employees.lastName
ORDER BY employees.lastName ASC;
""", conn)

,firstName,lastName,totalProductsSold
0,Loui,Bondur,6186
1,Larry,Bott,8205
2,Pamela,Castillo,9290
3,Julie,Firrelli,4227
4,Andy,Fixter,6246
5,Martin,Gerard,4180
6,Gerard,Hernandez,14231
7,Leslie,Jennings,11854
8,Barry,Jones,7486
9,Peter,Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [46]:
# Your code here
pd.read_sql("""
SELECT employees.firstName, employees.lastName, COUNT(DISTINCT orderdetails.productCode) AS uniqueProductsSold
FROM employees
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders ON customers.customerNumber = orders.customerNumber
JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber
GROUP BY employees.employeeNumber, employees.firstName, employees.lastName
HAVING COUNT(DISTINCT orderdetails.productCode) > 200
ORDER BY employees.lastName ASC;
""", conn)

,firstName,lastName,uniqueProductsSold


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!